In [ ]:
#checking some things

from dwave.system.samplers import DWaveSampler
sampler = DWaveSampler()
print("Connected to sampler", sampler.solver.name)

In [ ]:
print("Maximum anneal schedule points: {}".format(sampler.properties["max_anneal_schedule_points"]))

In [ ]:
annealing_range = sampler.properties["annealing_time_range"]
max_slope = 1.0/annealing_range[0]
print("Annealing time range: {}".format(sampler.properties["annealing_time_range"]))
print("Maximum slope:", max_slope)

In [ ]:
#One unit cell from Zhou paper containing 4 stars. Setting eta=0

#couplings
J={
    #top right star
    (8,12):-1, (8,13):1, (8,14):1, (8,15):1,
    (9,12):1, (9,13):-1, (9,14):1, (9,15):1,
    (10,12):1, (10,13):1, (10,14):-1, (10,15):1,
    (11,12):1, (11,12):1, (11,12):1, (11,12):-1,
    
    #bottom left star
    (16,20):-1, (16,21):1, (16,22):1, (16,23):1,
    (17,20):1, (17,21):-1, (17,22):1, (17,23):1,
    (18,20):1, (18,21):1, (18,22):-1, (18,23):1,
    (19,20):1, (19,21):1, (19,22):1, (19,23):-1,
    
    #ferro coupling to ghost spins to gauge spin
    (12,4):1, (4,0): 1, (0,16): 1,
    (13,5):1, (5,1): 1, (1,17): 1,
    (14,6):1, (6,2): 1, (2,18): 1,
    (15,7):1, (7,3): 1, (3,19): 1,
}


#spins
h={
    #top left star - ghost spins
    0:1.0, 1:1.0, 2:1.0, 3:1.0,
    4:1.0, 5:1.0, 6:1.0, 7:1.0,
    
    #top right star
    8:1.0, 9:1.0, 10:1.0, 11:1.0,
    12:1.0, 13:1.0, 14:1.0, 15:1.0,
    
    #bottom left star
    16:1.0, 17:1.0, 18:1.0, 19:1.0,
    20:1.0, 21:1.0, 22:1.0, 23:1.0,
    
    #bottom right star
    24:0.0, 25:0.0, 26:0.0, 27:0.0, 
    28:0.0, 29:0.0, 30:0.0, 31:0.0, 
}

In [ ]:

from helpers.embedding import DirectEmbeddingComposite

sampler_embedded = DirectEmbeddingComposite(sampler)

In [ ]:
#running with default anneal schedule
runs=1000
results = sampler_embedded.sample_ising(h, J, 
    num_reads=runs,
    answer_mode='raw',
    label='Notebook - Anneal Schedule',
    annealing_time=100)

print("QPU time used:", results.info['timing']['qpu_access_time'], "microseconds.")

In [ ]:
#histogram
plt.hist(results.record.energy,rwidth=1,align='left',bins=[-21,-20,-19,-18,-17,-16,-15])
plt.show()

In [ ]:
_, counts = np.unique(results.record.energy.reshape(1000,1), axis=0, return_counts=True)
print("Ground state probability: ", counts[0]/runs)